爬取携程网实时火车班次数据步骤  
1.获取城市缩写数据字典  
车站名获取url：https://kyfw.12306.cn/otn/resources/js/framework/station_name.js   
2.循环所有城市获取所有火车班次 票务数据（翻页） 获取车次、时间、始发站、终点站   
时刻表获取url：http://trains.ctrip.com/TrainBooking/Ajax/SearchListHandler.ashx?Action=getIntermediateStopList  
3.根据车次、时间、始发站、终点站获取火车时刻表  
导出方式优化 运行方式优化  
TODO:结果存入数据库,打包可在服务器运行,优化调用逻辑,更好快地获取车次

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 2 14:15:22 2020

@author: MarissaMan
"""

import requests
import json
import re
import pandas as pd
import csv
from bs4 import BeautifulSoup
import datetime
import time
import numpy as np


In [2]:
# 步骤一
# 获取12306所有火车站点名称  2992个站点
size = 0
train_dict = []

url='https://kyfw.12306.cn/otn/resources/js/framework/station_name.js'
response=requests.get(url).text
#去掉文本总最后多余的两个符号，并以@符号进行分割，第一项不是有用的信息
data=response[:-2].split("@")[1:]
for info in data:
    station_list=info.split("|")
    #将车站的汉字，全拼，简拼,代码取出
    train_dict.append([station_list[1],station_list[3],station_list[4],station_list[2]])
stationS = pd.DataFrame(train_dict,columns=["车站名","全拼","简拼","代码"])
print(str(len(train_dict))+'个站点')
stationS.to_csv('火车站点数据_'+str(len(train_dict))+'个站点_'+str(datetime.datetime.now()).split(" ")[0]+'.csv',index=None)

# 携程网车站名称(废弃)
# indexes = ["ABCD", "EFGH", "JKLM", "NOPQRS", "TUVWX", "YZ"]

# url='https://webresource.ctrip.com/ResTrainOnline/R3/TrainBooking/JS/stationbus_gb2312.js?2019_9_23_17_16_35'
# response=requests.get(url=url,timeout=10)
# # print(response.text)
# response = r.text.split(";")[1].replace(" cQuery.jsonpResponse.suggestion = ","")
# data = json.loads(response.replace('display','"display"').replace('data','"data"'))

# for i in indexes:
#     tmpData = data.get(i)
#     size = size + len(tmpData)
#     for row in tmpData:
#         try:
#             if row.get("data") == None:
#                 continue
#             tmp = row.get("data").split("|")
#             train_dict.append([tmp[1],tmp[2],i])
#         except:
#             continue
# print(str(size)+'个站点')
# 
# stationS = pd.DataFrame(train_dict,columns=["车站名","车站名_拼音",'拼音类型'])
# stationS.to_csv('火车站点数据_'+str(size)+'个站点_'+str(datetime.datetime.now()).split(" ")[0]+'.csv',index=None)

2992个站点


In [3]:
#步骤二
# 拼装查询参数
# 输入参数：查询时间段 查询车站名 所有车站名 stationS 
# 输出数据：查询参数数组 stationAndTimeS

# stationS = pd.read_csv("火车站点数据_2992个站点_2020-02-02.csv")
queryStation = ["阳江","阳春","阳西","阳东"]
queryDate = ["2020-02-03"]
stationAndStation = pd.DataFrame(columns=["DepartureCityName","ArrivalCityName"])
stationAndTimeS = pd.DataFrame(columns=["DepartureCityName","ArrivalCityName","date"])
size = len(stationS["车站名"])
print("所有车站数:"+str(len(stationS["车站名"])))
for q in stationS[stationS["车站名"].isin(queryStation)]["车站名"]:
    tmp = stationS[["车站名"]]
    tmp['终点站'] = q
    tmp.columns = ["DepartureCityName","ArrivalCityName"]
    stationAndStation = stationAndStation.append(tmp, ignore_index=True)
stationAndStation = stationAndStation[~(stationAndStation["DepartureCityName"]==stationAndStation["ArrivalCityName"])]
print("单程查询数量:"+str(stationAndStation.shape[0]))
stationAndStation = stationAndStation.append(stationAndStation[["DepartureCityName","ArrivalCityName"]])
print("双程查询数量:"+str(stationAndStation.shape[0]))

for date in queryDate:
    stationAndStation["date"]=date
    stationAndTimeS = stationAndTimeS.append(stationAndStation, ignore_index=True)

所有车站数:2992
单程查询数量:11964
双程查询数量:23928


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
# 步骤二
# 根据始发站、终点站、日期获取所有火车票信息
# 输入数据：始发站、终点站、日期 stationAndTimeS
# 输出数据：所有车票数据 ticketDetail
ticketDetail = []
#直达和非直达的区别要注意
def post_html(DepartureCityName,ArrivalCityName,date):
    try:
        url = 'http://trains.ctrip.com/TrainBooking/Ajax/SearchListHandler.ashx?Action=getSearchList'
        headers = {
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        }
        data = {
             'value':'{\
            "DepartureCityName":"'+DepartureCityName+'","ArrivalCityName":"'+ArrivalCityName+'","DepartureDate":"'+date+'",\
            }'
#             'value':'{"IsBus":false,"Filter":"0","Catalog":"","IsGaoTie":false,"IsDongChe":false,"CatalogName":"","HubCity":"",\
#             "DepartureCityName":"'+DepartureCityName+'","ArrivalCityName":"'+ArrivalCityName+'","DepartureDate":"'+date+'",\
#             "DepartureDateReturn":"","ArrivalDate":"","TrainNumber":""}'
        }
        response = requests.post(url=url,data=data ,headers=headers, timeout=60)
        response.encoding = response.apparent_encoding
        html = response.text
        # print(html)
        html = json.loads(html)
 
        return html
    except:
        return None
def parse_html(html,queryDate):
    try:
        info_list = []
        for i in html['TrainItemsList']:
            info = []
            TrainName = i['TrainName']#车次
            StartStationName = i['StartStationName']#始发站
            EndStationName = i['EndStationName']#终点站
#             StartTime = i['StratTime']#到达时间
#             EndTime = i['EndTime']#发车时间
#             TakeTime = i['TakeTime']#车程时间
            info.append(TrainName)
            info.append(StartStationName)
            info.append(EndStationName)
#             info.append(StartTime)
#             info.append(EndTime)
#             info.append(TakeTime)
            info.append(queryDate)
#             for j in i['SeatBookingItem']:
#                 SeatName_Price = j['SeatName']+'|'+ j['Price']
#                 info.append(SeatName_Price.replace('\\u3000',''))
            info_list.append(info)
        return info_list
    except:
        return []
# 模拟数据
# stationAndTimeS = pd.DataFrame([["阳江","广州南","2020-02-03"],["阳江","广州南","2020-02-05"]],columns=["DepartureCityName","ArrivalCityName","date"])
for row in stationAndTimeS.values:
    html = post_html(row[0],row[1],row[2])
    # print(html)
    if html == None:
        continue
    info_list = parse_html(html,row[2])
    print(len(info_list))
    # print(info_list)
    ticketDetail = ticketDetail+info_list
# write2csv(info_list)
trainNumS = pd.DataFrame(ticketDetail,columns=["trainNum","DepartureCityName","ArrivalCityName","date"])
trainNumS.to_csv('火车车次数据'+str(datetime.datetime.now()).split(" ")[0]+'.csv',index=None)
trainNumS
# def write2csv(info_list):
#     with open('班次信息.csv','a',encoding='utf-8-sig',newline='') as f:
#         writer = csv.writer(f)
#         for i in info_list:
#             writer.writerow(i)
 
# if __name__ == '__main__':
# with open('city.csv','r',encoding='utf-8-sig') as e:
#     reader_e = list(csv.reader(e))
#     for DepartureCityName in reader_e:
#         # print(DepartureCityName[0])
#         for ArrivalCityName in reader_e:
#             print(DepartureCityName[0]+'-'+ArrivalCityName[0])
#             if DepartureCityName == ArrivalCityName:
#                 continue
#             html = post_html(url,DepartureCityName,ArrivalCityName)
#             if html == None:
#                 continue
#             info_list = parse_html(html)
#             write2csv(info_list)

0
0
0
0
0
5
0
0
0
0
5
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5
5
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
2
2
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0


In [6]:
# 步骤三
# 运行数据准备：车次、日期、始发站、终点站 上一步输出：trainNumS
# 输出数据：车次时刻表
trainTable = []

#根据车次、始发站、终点站、日期获取车次时刻表数据
def post_trainTable(trainNum,DepartureCityName,ArrivalCityName,date):
    try:
        url = 'http://trains.ctrip.com/TrainBooking/Ajax/SearchListHandler.ashx?Action=getIntermediateStopList'
        headers = {
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8'
        }
        data = {
            'value':'{"trainNum":"'+trainNum+'","departure":"'+DepartureCityName+'","arrive":"'+ArrivalCityName+'","date":"'+date+'"}'
        }
        response = requests.post(url=url,data=data ,headers=headers, timeout=60)
        response.encoding = response.apparent_encoding
        html = json.loads(response.text)
 
        return html
    except:
        return None

# 循环获取所有车次数据
# 模拟数据
# trainNumS = pd.DataFrame([["C7208","阳江","广州南","2020-02-03"]],columns=["trainNum","DepartureCityName","ArrivalCityName","date"])
for row in trainNumS.values:
    html = post_trainTable(row[0],row[1],row[2],row[3])
    if html == None:continue
    soup = BeautifulSoup(html,"html.parser",from_encoding="utf-8").find_all("dl")
    
    for i in soup :
        t = list([row[0],row[3],str(i.find('dt',class_='d1').get_text())\
                ,str(i.find('dd',class_='d2').get_text())\
                ,str(i.find('dd',class_='d3').get_text())\
                ,str(i.find('dd',class_='d4').get_text())\
                ,str(i.find('dd',class_='d5').get_text())\
        ])
        trainTable.append(t);
trainTable = pd.DataFrame(trainTable,columns=["车次","日期","编号","车站名","到达时间","发车时间","停留时间"])
trainTable.to_csv('火车时刻表数据'+str(datetime.datetime.now()).split(" ")[0]+'.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [ ]:

# # 获取机场
# # 固定提供的 api 接口
# url = "https://flights.ctrip.com/itinerary/api/poi/get"
# # 根据自己的请求头修改
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0",
#     "Referer": "https://flights.ctrip.com/itinerary",
#     "Content-Type": "application/json"
# }

# # get请求
# response = requests.get(url, headers=headers).text
# print(response)

# def getData(Data, letters):
#     # 求长度，等下会用到的下标
#     length = len(letters)
#     # 得到各字母串的 json 信息
#     print("before")
#     letter = Data.get(letters)
#     print(letter)
#     # 挨个字母城市信息
#     for i in range(0, length):
#         # 字母 I 中没有东西，所以需要判断一下去掉
#         if letter.get(letters[i]) is not None:
#             # 每个城市信息
#             character = letter.get(letters[i])
#             for info in character:
#                 # 取自己想要的信息
#                 CN = info.get('display')
#                 # 取 左移三位到结尾的信息（即城市缩写）
#                 cityData = info.get('data')
#                 # 取编号
#                 EN = cityData[-3:]
#                 num = re.findall(r"\d+", cityData)
#                 # 不换行输出 格式为：（'阿勒泰': 'AAT' ）
#                 # print("'"+CN+"'"+":"+" '"+EN+"'"+", ", end="")
#                 print(CN, EN, num[0])
#                 # print(num)
#                 # 增加至数据库
#                 # sql = "insert into city(CN, EN, num) values('%s','%s',%d)" % (CN, EN, int(num[0]))
#                 # mysql(sql)
#     print("")
    
# # get请求
# response = requests.get(url, headers=headers).text
# # 以下都是 json 请求
# # print(response)
# data = json.loads(response).get('data')
# # print(data)
# getData(data, "ABCDEF")
# getData(data, "GHIJ")
# getData(data, "KLMN")
# # 少了一个字母O
# getData(data, "PQRSTUVW")
# getData(data, "XYZ")

In [ ]:
# # -*- coding: utf-8 -*-

# """
# Usage:
#     python hack12306.py [-c] <configpath>
# Options:
#     -h,--help   显示帮助菜单
#     -c          指定config.ini绝对路径
# Example:
#     python hack12306.py -c /usr/local/services/config.ini
#     或者
#     python hack12306.py
# """

# from splinter.browser import Browser
# from configparser import ConfigParser
# from time import sleep
# import traceback
# import time, sys
# import codecs
# import argparse
# import os
# import time

In [ ]:
# class hackTickets():
#     """docstring for hackTickets"""

#     """读取配置文件"""
#     def readConfig(self, config_file='config.ini'):
#         print("加载配置文件...")
#         # 补充文件路径，获得config.ini的绝对路径，默认为主程序当前目录
#         path = os.path.join(os.getcwd(), config_file)

#         cp = ConfigParser()
#         try:
#             # 指定读取config.ini编码格式，防止中文乱码（兼容windows）
#             cp.readfp(codecs.open(config_file, "r", "utf-8-sig"))
#         except IOError as e:
#             print(u'打开配置文件"%s"失败, 请先创建或者拷贝一份配置文件config.ini' % (config_file))
#             input('Press any key to continue')
#             sys.exit()
#         # 登录名
#         self.username = cp.get("login", "username")
#         # 密码
#         self.passwd = cp.get("login", "password")
#         # 始发站
#         starts_city = cp.get("cookieInfo", "starts")
#         # config.ini配置的是中文，转换成"武汉,WHN"，再进行编码
#         self.starts = self.convertCityToCode(starts_city).encode('unicode_escape').decode("utf-8").replace("\\u", "%u").replace(",", "%2c")
#         # 终点站
#         ends_city = cp.get("cookieInfo", "ends");
#         self.ends = self.convertCityToCode(ends_city).encode('unicode_escape').decode("utf-8").replace("\\u", "%u").replace(",", "%2c")
#         # 乘车时间
#         self.dtime = cp.get("cookieInfo", "dtime")
#         # 车次
#         orderStr = cp.get("orderItem", "order")
#         # 配置文件中的是字符串，转换为int
#         self.order = int(orderStr)
#         # 乘客名
#         self.users = cp.get("userInfo", "users").split(",")
#         # 车次类型
#         self.train_types = cp.get("trainInfo", "train_types").split(",")
#         # 发车时间
#         self.start_time = cp.get("trainInfo", "start_time")
#         # 网址
#         self.ticket_url = cp.get("urlInfo", "ticket_url")
#         self.login_url = cp.get("urlInfo", "login_url")
#         self.initmy_url = cp.get("urlInfo", "initmy_url")
#         self.buy = cp.get("urlInfo", "buy")

#         # 席别
#         seat_type = cp.get("confirmInfo", "seat_type")
#         self.seatType = self.seatMap[seat_type] if seat_type in self.seatMap else ""

#         # 是否允许分配无座
#         noseat_allow = cp.get("confirmInfo", "noseat_allow")
#         self.noseat_allow = 1 if int(noseat_allow) != 0 else 0

#         # 浏览器名称：目前使用的是chrome
#         self.driver_name = cp.get("pathInfo", "driver_name")
#         # 浏览器驱动（目前使用的是chromedriver）路径
#         self.executable_path = cp.get("pathInfo", "executable_path")

#     def loadConfig(self):
#         parser = argparse.ArgumentParser()
#         parser.add_argument('-c', '--config', help='Specify config file, use absolute path')
#         args = parser.parse_args()
#         if args.config:
#             # 使用指定的配置文件
#             self.readConfig(args.config)
#         else:
#             # 使用默认的配置文件config.ini
#             self.readConfig()

#     """
#         加载映射文件，并将中文"武汉"转换成编码后的格式：“武汉,WHN“
#     """
#     def loadCityCode(self):
#         print("映射出发地、目的地...")
#         city_codes = {}
#         path = os.path.join(os.getcwd(), 'city_code.txt')
#         print("path:"+str(path))
#         with codecs.open(path, "r", "utf-8-sig") as f:
#             for l in f.readlines():
#                 city = l.split(':')[0]
#                 code = l.split(':')[1].strip()
#                 city_codes[city] = city + "," + code
#         return city_codes

#     def convertCityToCode(self, c):
#         try:
#             return self.city_codes[c]
#         except KeyError:
#             print("转换城市错误，请修改config.ini中starts或者ends值为中文城市名")
#             return False

#     """加载席别编码"""
#     def loadSeatType(self):
#         self.seatMap = {
#             "硬座" : "1",
#             "硬卧" : "3",
#             "软卧" : "4",
#             "一等软座" : "7",
#             "二等软座" : "8",
#             "商务座" : "9",
#             "一等座" : "M",
#             "二等座" : "O",
#             "混编硬座" : "B",
#             "特等座" : "P"
#         }

#     def __init__(self):
#         # 读取城市中文与三字码映射文件，获得转换后到城市信息-- “武汉”: "武汉,WHN"
#         self.city_codes = self.loadCityCode();

#         # 加载席别
#         self.loadSeatType()

#         # 读取配置文件，获得初始化参数
#         self.loadConfig();

#     def login(self):
#         print("开始登录...")
#         # 登录
#         self.driver.visit(self.login_url)
#         # 自动填充用户名
#         self.driver.fill("loginUserDTO.user_name", self.username)
#         # 自动填充密码
#         self.driver.fill("userDTO.password", self.passwd)

#         print(u"等待验证码，自行输入...")

#         # 验证码需要自行输入，程序自旋等待，直到验证码通过，点击登录
#         while True:
#             if self.driver.url != self.initmy_url:
#                 sleep(1)
#             else:
#                 break

#     """更多查询条件"""
#     def searchMore(self):
#         # 选择车次类型
#         for type in self.train_types:
#             # 车次类型选择
#             train_type_dict = {'T': u'T-特快',                # 特快
#                                 'G': u'GC-高铁/城际',         # 高铁
#                                 'D': u'D-动车',               # 动车
#                                 'Z': u'Z-直达',               # 直达
#                                 'K': u'K-快速'                # 快速
#                                 }
#             if type == 'T' or type == 'G' or type == 'D' or type == 'Z' or type == 'K':
#                 print(u'--------->选择的车次类型', train_type_dict[type])
#                 self.driver.find_by_text(train_type_dict[type]).click()
#             else:
#                 print(u"车次类型异常或未选择!(train_type=%s)" % type)

#         # 选择发车时间
#         print(u'--------->选择的发车时间', self.start_time)
#         if self.start_time:
#             self.driver.find_option_by_text(self.start_time).first.click()
#         else:
#             print(u"未指定发车时间，默认00:00-24:00")

#     """填充查询条件"""
#     def preStart(self):
#         # 加载查询信息
#         # 出发地
#         self.driver.cookies.add({"_jc_save_fromStation": self.starts})
#         # 目的地
#         self.driver.cookies.add({"_jc_save_toStation": self.ends})
#         # 出发日
#         self.driver.cookies.add({"_jc_save_fromDate": self.dtime})

#     def specifyTrainNo(self):
#         count=0
#         while self.driver.url == self.ticket_url:
#             # 勾选车次类型，发车时间
#             self.searchMore();
#             sleep(0.05)
#             self.driver.find_by_text(u"查询").click()
#             count += 1
#             print(u"循环点击查询... 第 %s 次" % count)

#             try:
#                 self.driver.find_by_text(u"预订")[self.order - 1].click()
#                 sleep(0.3)
#             except Exception as e:
#                 print(e)
#                 print(u"还没开始预订")
#                 continue

#     def buyOrderZero(self):
#         count=0
#         while self.driver.url == self.ticket_url:
#             # 勾选车次类型，发车时间
#             self.searchMore();
#             sleep(0.05)
#             self.driver.find_by_text(u"查询").click()
#             count += 1
#             print(u"循环点击查询... 第 %s 次" % count)

#             try:
#                 for i in self.driver.find_by_text(u"预订"):
#                     i.click()
#                     # 等待0.3秒，提交等待的时间
#                     sleep(0.3)

#             except Exception as e:
#                 print(e)
#                 print(u"还没开始预订 %s" %count)
#                 continue

#     def selUser(self):
#         print(u'开始选择用户...')
#         for user in self.users:
#             self.driver.find_by_text(user).last.click()

#     def confirmOrder(self):
#         print(u"选择席别...")
#         if self.seatType:
#             self.driver.find_by_value(self.seatType).click()
#         else:
#             print(u"未指定席别，按照12306默认席别")

#     def submitOrder(self):
#         print(u"提交订单...")
#         sleep(1)
#         self.driver.find_by_id('submitOrder_id').click()

#     def confirmSeat(self):
#         # 若提交订单异常，请适当加大sleep的时间
#         sleep(1)
#         print(u"确认选座...")
#         if self.driver.find_by_text(u"硬座余票<strong>0</strong>张") == None:
#             self.driver.find_by_id('qr_submit_id').click()
#         else:
#             if self.noseat_allow == 0:
#                 self.driver.find_by_id('back_edit_id').click()
#             elif self.noseat_allow == 1:
#                 self.driver.find_by_id('qr_submit_id').click()

# #     def buyTickets(self):
# #         t = time.clock()
# #         try:
# #             print(u"购票页面开始...")

# #             # 填充查询条件
# #             self.preStart()

# #             # 带着查询条件，重新加载页面
# #             self.driver.reload()

# #             # 预定车次算法：根据order的配置确定开始点击预订的车次，0-从上至下点击，1-第一个车次，2-第二个车次，类推
# #             if self.order != 0:
# #                 # 指定车次预订
# #                 self.specifyTrainNo()
# #             else:
# #                 # 默认选票
# #                 self.buyOrderZero()
# #             print(u"开始预订...")

# #             sleep(0.8)
# #             # 选择用户
# #             self.selUser()
# #             # 确认订单
# #             self.confirmOrder()
# #             # 提交订单
# #             self.submitOrder()
# #             # 确认选座
# #             self.confirmSeat()

# #             print(time.clock() - t)

# #         except Exception as e:
# #             print(e)

#     """入口函数"""
#     def start(self):
#         # 初始化驱动
#         self.driver=Browser(driver_name=self.driver_name,executable_path=self.executable_path)
#         # 初始化浏览器窗口大小
#         self.driver.driver.set_window_size(1400, 1000)

#         # 登录，自动填充用户名、密码，自旋等待输入验证码，输入完验证码，点登录后，访问 tick_url（余票查询页面）
#         print("start login")
#         self.login()

#         # 登录成功，访问余票查询页面
# #         self.driver.visit(self.ticket_url)

#         # 自动购买车票
# #         self.buyTickets();

# # if __name__ == '__main__':
# #     print("===========hack12306 begin===========")
# #     hackTickets = hackTickets()
# #     hackTickets.start()

In [ ]:
# # 获取所有车站数据字典
# def getStation():
#     # 12306的城市名和城市代码js文件url
#     url = 'https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9018'
#     r = requests.get(url, verify=False)
#     pattern = u'([\u4e00-\u9fa5]+)\|([A-Z]+)'
#     result = re.findall(pattern, r.text)
#     print(result)
#     station = dict(result)  # 所有车站信息
#     #print(station)
#     return result
# tmp = pd.DataFrame(str(station).strip().replace("[","").replace("]","").replace("(","").replace("'","").replace("),","|").replace(")","").replace(",",":").split("|"))
# tmp.to_csv("test1.csv",index=None,columns=None)
# tmp

In [ ]:
# # coding: utf-8

# """命令行火车票查看器
# Usage:
#     ticket [-gdtkz] <from> <to> <date>
# Options:
#     -h,--help   显示帮助菜单
#     -g          高铁
#     -d          动车
#     -t          特快
#     -k          快速
#     -z          直达
# Example:
#     tickets 北京 上海 2016-10-10
#     tickets -dg 成都 南京 2016-10-10
# """

# from docopt import docopt
# import requests
# import prettytable
# from colorama import init, Fore
# from stations import stations


# def cli():
#     """command-line interface"""
#     arguments = docopt(__doc__)
#     from_station = stations.get(arguments['<from>'])
#     to_station = stations.get(arguments['<to>'])
#     date = arguments['<date>']
#     # stated_url=u'https://kyfw.12306.cn/otn/leftTicket/queryO?leftTicketDTO.train_date=2018-3-28&leftTicketDTO.from_station=CDW&leftTicketDTO.to_station=CQW&purpose_codes=ADULT'
#     URL = (
#     'https://kyfw.12306.cn/otn/leftTicket/queryO?leftTicketDTO.train_date={}&leftTicketDTO.from_station={}&leftTicketDTO.to_station={}&purpose_codes=ADULT').format(
#         date, from_station, to_station
#     )
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
#         , 'Connection': 'keep-alive'}

#     r = requests.get(URL, verify=False, headers=headers)
#     available_trains = r.json()['data']['result']
#     available_place = r.json()['data']['map']
#     options = ''.join([
#         key for key, value in arguments.items() if value is True
#     ])
#     TrainsCollection(available_trains, available_place, options).pretty_print()


# class TrainsCollection:
#     header = '车次 车站 时间 历时 一等 二等 高级卧铺 软卧 硬卧 硬座 无座'.split()

#     def __init__(self, available_trains, available_place, options):
#         """查询的火车班次集合
#         :param available_trains:一个列表，包含可获得的火车班次，每个火车班次是一个字典
#         :param options:查询的选项，如高铁，动车，等等"""

#         self.available_trains = available_trains
#         self.available_place = available_place
#         self.options = options

#     @property
#     def trains(self):
#         for raw_train in self.available_trains:
#             raw_train_list = raw_train.split('|')
#             train_no = raw_train_list[3]
#             initial = train_no[0].lower()
#             duration = raw_train_list[10]
#             init()
#             if not self.options or initial in self.options:
#                 train = [
#                     train_no,#train number
#                     '\n'.join([Fore.LIGHTGREEN_EX + self.available_place[raw_train_list[6]] + Fore.RESET,#始发站
#                                Fore.LIGHTRED_EX + self.available_place[raw_train_list[7]] + Fore.RESET]),#终点站
#                     '\n'.join([Fore.LIGHTGREEN_EX + raw_train_list[8] + Fore.RESET,
#                                Fore.LIGHTRED_EX + raw_train_list[9] + Fore.RESET]),
#                     duration,#时长
#                     raw_train_list[-6] if raw_train_list[-6] else '--',#一等
#                     raw_train_list[-7] if raw_train_list[-7] else '--',  # 二等
#                     raw_train_list[-15] if raw_train_list[-15] else '--',  # 高级软卧
#                     raw_train_list[-8] if raw_train_list[-8] else '--',  # 软卧
#                     raw_train_list[-14] if raw_train_list[-14] else '--',  # 硬卧
#                     raw_train_list[-11] if raw_train_list[-11] else '--',  # 硬座
#                     raw_train_list[-9] if raw_train_list[-9] else '--',  # 无座
#                 ]
#                 yield  train

#     def pretty_print(self):
#         pt = prettytable.PrettyTable()
#         pt._set_field_names(self.header)
#         for train in self.trains:
#             pt.add_row(train)
#         print(pt)


# if __name__ == '__main__':
#     cli()

In [ ]:
# # -*- coding: utf-8 -*-
# # ‘‘‘
# # 获取12306城市名和城市代码的数据
# # 文件名： parse_station.py
# # ‘‘‘
# import requests
# import re
# import json
# import pandas as pd

# #关闭https证书验证警告
# requests.packages.urllib3.disable_warnings()

In [ ]:
# class Login():
#     # """实现12306的登录"""
#     def __init__(self):
#         self.capycha_url = 'https://kyfw.12306.cn/passport/captcha/captcha-image?login_site=E&module=login&rand=sjrand&0.27575294592849064'

#         self.capycha_check_url = 'https://kyfw.12306.cn/passport/captcha/captcha-check'

#         self.login_url = 'https://kyfw.12306.cn/passport/web/login';
#     def download_code(self):

#     # """1.下载验证码"""

#         response = session.get(self.capycha_url)

#         f = open('code.jpg','wb')

#         f.write(response.content)

#         f.close()

#         self.check_code()

In [ ]:
# def getStation():
#     # 12306的城市名和城市代码js文件url
#     url = 'https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9018';
#     r = requests.get(url,verify=False)
#     pattern = u'([\u4e00-\u9fa5]+)\|([A-Z]+)';
#     result = re.findall(pattern,r.text)
#     station = dict(result)#{'北京北': ‘VAP‘, ‘北京东‘: ‘BOP‘, ‘北京‘: ‘BJP‘,
#     #print(station)
#     return station
# station = getStation();

In [ ]:
# sttt = str(station)
# tmp = pd.DataFrame(sttt.replace('{','').replace('}','').replace('\'','').split(','));
# tmp['name'] = tmp[0].map(lambda x : x.split(':')[0]);
# tmp['code'] = tmp[0].map(lambda x : x.split(':')[1]);
# tmp.to_excel('所有火车站点名称.xlsx');
# tmp

In [ ]:
# # '''
# # 查询两站之间的火车票信息
# # 输入参数： <date> <from> <to>
# # 12306 api:
# # 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=2017-07-18&leftTicketDTO.from_station=NJH&leftTicketDTO.to_station=SZH&purpose_codes=ADULT'
# # '''

# #生成查询的url
# def get_query_url(text):
#     # 城市名代码查询字典
#     # key：城市名 value：城市代码
#     try:
#         date = '2020-02-01'
#         from_station_name = '上海'
#         to_station_name = '北京'
#         from_station = text[from_station_name]
#         to_station = text[to_station_name]
#     except:
#         date, from_station, to_station = '--', '--', '--'
#         # 将城市名转换为城市代码

#     # api url 构造
#     url = (
#     'https://kyfw.12306.cn/otn/leftTicket/query?'
#     'leftTicketDTO.train_date={}&'
#     'leftTicketDTO.from_station={}&'
#     'leftTicketDTO.to_station={}&'
#     'purpose_codes=ADULT'
#     ).format(date, from_station, to_station)
#     print(url)
    
#     return url

In [ ]:
# stationsA = ['广州南','阳江','湛江西','珠海','茂名','佛山西','北京西','汕头','长沙南','昆明','广州','上海南','深圳西','深圳东','昭通','信宜','阳东','阳西','阳春','肇庆东','深圳北','海口','成都'];
# stationsB = ['阳春','阳东','阳江','阳西']
# get_query_url(station)
# tmp[tmp['name'].str.contains('阳\.')==True]
# url = get_query_url("");
# url

In [ ]:
# https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=--&leftTicketDTO.from_station=--&leftTicketDTO.to_station=--&purpose_codes=ADULT
# 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=--&leftTicketDTO.from_station=--&leftTicketDTO.to_station=--&purpose_codes=ADULT'




# #获取信息
# def query_train_info(url,text):
# ‘‘‘
# 查询火车票信息：
# 返回 信息查询列表
# ‘‘‘

# info_list = []
# try:
# r = requests.get(url, verify=False)

# # 获取返回的json数据里的data字段的result结果
# raw_trains = r.json()[‘data‘][‘result‘]

# for raw_train in raw_trains:
# # 循环遍历每辆列车的信息
# data_list = raw_train.split(‘|‘)

# # 车次号码
# train_no = data_list[3]
# # 出发站
# from_station_code = data_list[6]
# from_station_name = text[‘上海‘]
# # 终点站
# to_station_code = data_list[7]
# to_station_name = text[‘北京‘]
# # 出发时间
# start_time = data_list[8]
# # 到达时间
# arrive_time = data_list[9]
# # 总耗时
# time_fucked_up = data_list[10]
# # 一等座
# first_class_seat = data_list[31] or ‘--‘
# # 二等座
# second_class_seat = data_list[30]or ‘--‘
# # 软卧
# soft_sleep = data_list[23]or ‘--‘
# # 硬卧
# hard_sleep = data_list[28]or ‘--‘
# # 硬座
# hard_seat = data_list[29]or ‘--‘
# # 无座
# no_seat = data_list[26]or ‘--‘

# # 打印查询结果
# info = (‘车次:{}\n出发站:{}\n目的地:{}\n出发时间:{}\n到达时间:{}\n消耗时间:{}\n座位情况：\n 一等座：「{}」 \n二等座：「{}」\n软卧：「{}」\n硬卧：「{}」\n硬座：「{}」\n无座：「{}」\n\n‘.format(
# train_no, from_station_name, to_station_name, start_time, arrive_time, time_fucked_up, first_class_seat,
# second_class_seat, soft_sleep, hard_sleep, hard_seat, no_seat))


# print(info)
# info_list.append(info)

# return info_list
# except:
# return ‘ 输出信息有误，请重新输入‘


# text=getStation();
# url=get_query_url(text)

# query_train_info(url,text)

In [ ]:
# import re
# import json
# import shelve
# import requests
# import datetime
# # import prettytable as pt
# from colorama import Fore,init

# init(autoreset = False)

# #获取全部车站信息    
# with shelve.open("db") as db:#shelve序列化
#     stations={}
#     if not stations:
#         url="https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9102"
#         txt=requests.get(url).text   
#         inf=txt[:-2].split("@")[1:]
#         for record in inf:
#             rlist=record.split("|")
#             stations[(rlist[2])]={"cn":rlist[1],"id":rlist[2],"qp":rlist[3],"jp":rlist[4]}
#         db["stations"]=stations

In [ ]:
# # 车站信息
# def getcode(t):
#     while 1:
#         s1=input("%s站："%t)
#         r1=[]
#         for id,station in stations.items():
#             if s1 in station.values():
#                 r1.append((id,station))
#         if r1:
#             break
#         print("没有这个车站！")
#         print("请重新输入！")
#     if len(r1)==1:
#         sid=r1[0][0]
#     else:
#         #print(r1)
#         print("你输入的条件比较模糊，请在以下站中进行选择！")
#         for i in range(len(r1)):
#             print(i+1,r1[i][1]["cn"])#i代表所在元组，1代表元组后1位所对应的具体选项
#         sel=int(input("你的选择："))-1
#         sid=r1[sel][0]
#     return sid

# fromid=getcode("出发")
# toid=getcode("到达")

In [ ]:
# requests.packages.urllib3.disable_warnings()
# #日期
# def date():
#     while 1:
#         from_date=input("请输入出发日期(输入格式2019-01-01)：")
#         now=datetime.date.today()
#         last=now+datetime.timedelta(days=14)
#         tra_date=datetime.date(int(from_date[:4]),int(from_date[5:7]),int(from_date[8:]))
        
#         if tra_date>=now and tra_date<=last:
#             return tra_date
#             break
#         else:
#             print("输入错误，请输入未来15内天的日期进行查询！")
#             print("请重新输入！")
# from_date=date()
# tianshu=input("请输入要查询的天数（15天内）：").strip()
# for i in range(int(tianshu)):
#     train_date=from_date+datetime.timedelta(days=i)
#     qurl="https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date={}&leftTicketDTO.from_station={}&leftTicketDTO.to_station={}&purpose_codes=ADULT".format(from_date,fromid,toid)
#     print(qurl)
#     t = requests.get(qurl).json();
#     print(t)
#     json_result=requests.get(qurl).json()['data']['result']
    
 
#     print("您输入的查询条件是：出发站%s，到达站%s"%(stations[fromid]["cn"],stations[toid]["cn"]))
#     #ainf=requests.get(qurl).json()["data"]["result"]  
#     #车次信息  
#     #print(ainf[0],len(ainf))
    
#     result=[]
#     for i in json_result:
#         ilist=i.split("|")
#         checi=ilist[3]
#         chufa=stations[ilist[6]]["cn"]
#         daoda=stations[ilist[7]]["cn"]
#         ftime=ilist[8]
#         dtime=ilist[9]
#         ydeng=ilist[31]
#         edeng=ilist[30]
#         ywo=ilist[28]
#         rwo=ilist[23]
#         yzuo=ilist[29]
#         wzuo=ilist[26]
#         beizhu=ilist[1]
#         result.append((checi,chufa,daoda,ftime,dtime,ydeng,edeng,rwo,ywo,yzuo,wzuo,beizhu))
    
#     #print(result)
    
#     #车次信息制表打印   
#     def f1(alist):     
#         tb = pt.PrettyTable()
#         tb.junction_char = "*"
#         tb.field_names=["车次","出发站","到达站","出发时间","到达时间","一等座","二等座","软卧","硬卧","硬座","无座","备注"]
#         for i in alist:
#             tb.add_row(["%-5s"%i[0],Fore.LIGHTRED_EX+"%-5s"%i[1]+Fore.RESET,"%-5s"%i[2],Fore.LIGHTRED_EX+i[3]+Fore.RESET,i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11]])
#         print(tb)
 
#     #列车分车型    
#     gt=[]
#     lc=[]
#     for i in result:
#         if i[6]:
#             gt.append(i)
#         else:
#             lc.append(i)
 
#     #循环打印       
#     while 1:
#         cxing=input("请选择车型：1.高铁 2.列车 3.返回全部页面 4.退出\n")
#         if cxing=="1":
#             f1(gt)
#         elif cxing=="2":     
#             f1(lc)
#         elif cxing=="3":
#             f1(result)  
#         else :
#             break
# #             2020-01-15

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# import urllib
# qurl = 'https://kyfw.12306.cn/otn/leftTicket/queryZ?leftTicketDTO.train_date=2020-01-14&leftTicketDTO.from_station=WRQ&leftTicketDTO.to_station=WLQ&purpose_codes=ADULT';
# # t = requests.get(qurl)
# # headers = {'Cookie' : 'JSESSIONID=9FBDEB1BDE6A7C5555B44DECE1E5BADD; tk=FPsv2RT4-FAr2M4I-xtf7mdvS6kwXWxwA9TbQw09w1w0; route=495c805987d0f5c8c84b14f60212447d; BIGipServerotn=653263370.38945.0000; BIGipServerpool_passport=200081930.50215.0000; RAIL_EXPIRATION=1579262754097; RAIL_DEVICEID=by-B-vjAnGgvgIixiIN6AfhkXfOO9WBNvIk8Nfu0sPMe_61OquIPqFlhVg6Xj3JbQlH_PJyb3HTvKnYcTzoijHKRsUCwdmVr1PgpwywnqN28XfykJsd8-_zns-MQPrf5sQYc03K4wxS3E9ikpqn1HKjD6XsurLMC; _jc_save_fromStation=%u9633%u6C5F%2CWRQ; _jc_save_toStation=%u9633%u4E1C%2CWLQ; _jc_save_fromDate=2020-01-14; _jc_save_toDate=2020-01-14; _jc_save_wfdc_flag=dc'}
# # header = {
# # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
# # }
# # t = requests.get(qurl,headers);
# # # ['data']['result'];
# # soup = BeautifulSoup(t.content,'lxml')
# # soup
# # t.content


# req = urllib.request(qurl%(data,from_station,to_station))
# req.headers = headers
# html = opener.open(req).read()
# html

# urlopen



In [ ]:
# #!/usr/bin/env python
# #coding=utf8
# #12306查票爬虫
# import requests,json,sys
# #获取地址代码
# #https://kyfw.12306.cn/otn/resources/js/framework/favorite_name.js

# url = 'https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9025'
# url_tmp = str(requests.get(url, verify=False).content, encoding='utf-8').replace("var station_names ='@", '').replace(
#     "';", '')
# url_tmp = list(url_tmp.split('@'))
# def dizhi_code(dizhi):
#     '''
#     通过站点名字获取code
#     '''
#     for i in url_tmp:
#         if dizhi == i.split('|')[1]:
#             return i.split('|')[2]
# def code_dizhi(code):
#     '''
#     通过code获取站点名字
#     '''
# #    print (url_tmp)
#     for i in url_tmp:
#         if code == i.split('|')[2]:
#             return i.split('|')[1]

# def get_lieche(start_che,end_che,date):

#     start_che=dizhi_code(start_che)
#     end_che=dizhi_code(end_che)
#  #   print (start_che,end_che)
#     try:
#         url='https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=%s&leftTicketDTO.from_station=%s&leftTicketDTO.to_station=%s&purpose_codes=ADULT' %(date,start_che,end_che)
#         print (url)
#         data = json.loads(requests.get(url, verify=False).content)
#     except Exception as e:
#         print("获取数据失败，可能网络错误或者请求太频繁",e)
#         sys.exit(2)

#     chuli_data=[]
#     for i in data['data']['result']:
#         list_che=list(i.strip('|').split('|'))
#         if list_che[0]=="23:00-06:00系统维护时间" or list_che[0]=="预订" :
#                               #时间         列车班号    始发站                  终点站                  始发时间     终点时间    一共时间    商务座      一等座        二等座       软卧          硬卧
#             chuli_data.append((che_time,list_che[2],code_dizhi(list_che[3]),code_dizhi(list_che[4]),list_che[7],list_che[8],list_che[9],list_che[-4],list_che[-5],list_che[-6],list_che[-13],list_che[-8],list_che[-7]))
#         else:
#             chuli_data.append((che_time, list_che[3], code_dizhi(list_che[4]), code_dizhi(list_che[5]), list_che[8],
#                                list_che[9], list_che[10], list_che[-4], list_che[-5], list_che[-6], list_che[-13],list_che[-8],list_che[-7]))
# #        print (list_che)
#  #   print (chuli_data)
#     print("时间   \t\t始发站\t终点站\t始发时间\t终点时间\t一共时间\t商务座\t一等座\t二等座\t软卧\t硬卧\t硬座\t列车班号")
#     for i in chuli_data:
# #        print (i)
#         print("%s\t%s\t%s\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s" %(i[0],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11],i[12],i[1]))
# if __name__ == '__main__':
#     #get_lieche("重庆","北京",'2017-09-06')
#     start_zhan=input("请输入起始站：")
#     end_zhan = input("请输入终点站：")
#     che_time = input("请输入时间范例2017-09-03：")
#     if start_zhan=="" or end_zhan=="" or che_time=="":
#         print("站点名错误")
#         sys.exit(1)
#  #   elif che_time=="":
# #        che_time=datetime.date()

#     get_lieche(start_zhan, end_zhan, che_time)


In [ ]:
# # # coding: utf-8
# #
# import re
# import requests
# from pprint import pprint
# import time
# headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
#          , 'Connection':'keep-alive'}

# url = 'https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9050'

# response = requests.get(url=url, verify=False, headers = headers)

# stations = re.findall(u'([\u4e00-\u9fa5]+)\|([A-Z]+)', response.text)

# pprint(dict(stations),indent=4)

In [ ]:
# url='https://kyfw.12306.cn/otn/leftTicket/queryZ?leftTicketDTO.train_date=2020-01-14&leftTicketDTO.from_station=WRQ&leftTicketDTO.to_station=WLQ&purpose_codes=ADULT'
# # print (url)
# # data = json.loads(requests.get(url, verify=False).content)
# # data

# # URL = (
# # 'https://kyfw.12306.cn/otn/leftTicket/queryO?leftTicketDTO.train_date={}&leftTicketDTO.from_station={}&leftTicketDTO.to_station={}&purpose_codes=ADULT').format(
# # date, from_station, to_station
# # )
# headers = {
# 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
# , 'Connection': 'keep-alive'}

# r = requests.get(url, verify=False, headers=headers)
# print(r)
# available_trains = r.json()['data']['result']
# available_place = r.json()['data']['map']

In [ ]:
# # coding: utf-8

# """命令行火车票查看器
# Usage:
#     ticket [-gdtkz] <from> <to> <date>
# Options:
#     -h,--help   显示帮助菜单
#     -g          高铁
#     -d          动车
#     -t          特快
#     -k          快速
#     -z          直达
# Example:
#     tickets 北京 上海 2016-10-10
#     tickets -dg 成都 南京 2016-10-10
# """

# from docopt import docopt
# import requests
# import prettytable
# from colorama import init, Fore
# # from stations import stations


# def cli():
#     """command-line interface"""
#     arguments = docopt(__doc__)
#     from_station = stations.get(arguments['<from>'])
#     to_station = stations.get(arguments['<to>'])
#     date = arguments['<date>']
#     # stated_url=u'https://kyfw.12306.cn/otn/leftTicket/queryO?leftTicketDTO.train_date=2018-3-28&leftTicketDTO.from_station=CDW&leftTicketDTO.to_station=CQW&purpose_codes=ADULT'
#     URL = (
#     'https://kyfw.12306.cn/otn/leftTicket/queryO?leftTicketDTO.train_date={}&leftTicketDTO.from_station={}&leftTicketDTO.to_station={}&purpose_codes=ADULT').format(
#         date, from_station, to_station
#     )
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
#         , 'Connection': 'keep-alive'}

#     r = requests.get(URL, verify=False, headers=headers)
#     available_trains = r.json()['data']['result']
#     available_place = r.json()['data']['map']
#     options = ''.join([
#         key for key, value in arguments.items() if value is True
#     ])
#     TrainsCollection(available_trains, available_place, options).pretty_print()


# class TrainsCollection:
#     header = '车次 车站 时间 历时 一等 二等 高级卧铺 软卧 硬卧 硬座 无座'.split()

#     def __init__(self, available_trains, available_place, options):
#         """查询的火车班次集合
#         :param available_trains:一个列表，包含可获得的火车班次，每个火车班次是一个字典
#         :param options:查询的选项，如高铁，动车，等等"""

#         self.available_trains = available_trains
#         self.available_place = available_place
#         self.options = options

#     @property
#     def trains(self):
#         for raw_train in self.available_trains:
#             raw_train_list = raw_train.split('|')
#             train_no = raw_train_list[3]
#             initial = train_no[0].lower()
#             duration = raw_train_list[10]
#             init()
#             if not self.options or initial in self.options:
#                 train = [
#                     train_no,#train number
#                     '\n'.join([Fore.LIGHTGREEN_EX + self.available_place[raw_train_list[6]] + Fore.RESET,#始发站
#                                Fore.LIGHTRED_EX + self.available_place[raw_train_list[7]] + Fore.RESET]),#终点站
#                     '\n'.join([Fore.LIGHTGREEN_EX + raw_train_list[8] + Fore.RESET,
#                                Fore.LIGHTRED_EX + raw_train_list[9] + Fore.RESET]),
#                     duration,#时长
#                     raw_train_list[-6] if raw_train_list[-6] else '--',#一等
#                     raw_train_list[-7] if raw_train_list[-7] else '--',  # 二等
#                     raw_train_list[-15] if raw_train_list[-15] else '--',  # 高级软卧
#                     raw_train_list[-8] if raw_train_list[-8] else '--',  # 软卧
#                     raw_train_list[-14] if raw_train_list[-14] else '--',  # 硬卧
#                     raw_train_list[-11] if raw_train_list[-11] else '--',  # 硬座
#                     raw_train_list[-9] if raw_train_list[-9] else '--',  # 无座
#                 ]
#                 yield  train

#     def pretty_print(self):
#         pt = prettytable.PrettyTable()
#         pt._set_field_names(self.header)
#         for train in self.trains:
#             pt.add_row(train)
#         print(pt)


# if __name__ == '__main__':
#     cli()